# Домашнее задание 1 по теме: определение тональной окраски отзыва с помощью словаря

Импорт необходимых модулей.

In [248]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import time
import re
import random
from random import randint
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import collections
from sklearn.metrics import accuracy_score

Используем UserAgent, который создаёт имитацию браузера, что бы нас не поймали. Для этого же time.sleep(1).

In [234]:
ua = UserAgent(verify_ssl=False)

Функции для выкачивания отзывов и оценок (только 1/5, 2/5 и 5/5, чтобы можно было поделить на две группы) пользователей.

In [235]:
def page_parse(new_href):
    while new_href is not None:
        rest_url = main_url + new_href
        time.sleep(1)
        req = requests.get(rest_url, headers={'User-Agent': ua.random})
        rest = req.text
        soup = BeautifulSoup(rest, 'html.parser')
        new_href = soup.find('a', {'class': 'nav next ui_button primary cx_brand_refresh_phase2'})
        if new_href is not None:
            new_href = new_href['href']
        review_blocks = soup.find_all('div', {'class': 'ui_column is-9'})
        for r in review_blocks:
            com = comments(str(r))
            if com[1] == 5:
                good.append(com[0])
            elif com[1] == 1 or com[1] == 2:
                bad.append(com[0])

In [236]:
def comments(rev):
    soup = BeautifulSoup(rev, 'html.parser')
    text = soup.find('p', {'class': 'partial_entry'}).get_text()
    mark = soup.find('span')
    mark = mark['class'][1]
    mark = int(re.sub(r'bubble_(\d)0', r'\1', mark))
    return text, mark

С сайта https://www.tripadvisor.ru выкачиваем отзывы о тёх ресторанах сети "Тануки".

In [237]:
main_url = 'https://www.tripadvisor.ru'
good = []
bad = []
first = ['/Restaurant_Review-g298484-d3775519-Reviews-Tanuki-Moscow_Central_Russia.html',
        '/Restaurant_Review-g298484-d3236588-Reviews-Tanuki-Moscow_Central_Russia.html',
        '/Restaurant_Review-g298484-d3200244-Reviews-Tanuki-Moscow_Central_Russia.html']
for f in first:
    page_parse(f)

Всего положительных и отрицательных отзывов для составления списков — по 90. Делим выборки. Random, чтобы отзывы о разных ресторанах, оставленные в разное время, перемешались. 

In [238]:
random.shuffle(good_list)
random.shuffle(bad_list)
good_list = good[0:90]
bad_list = bad[0:90]
good_test = good[90:105]
bad_test = bad[90:105]

Токенизация, лемматизация, приведение к нижнему регистру.

In [239]:
morph = MorphAnalyzer()

In [240]:
def preproc(text):
    text = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    new_lst = []
    for elem in text:
        new_lst.append(morph.parse(elem)[0].normal_form)
    return new_lst

Составление двух списков уникальных слов из отзывов положительной и отрицательной групп. Фильтр отсеивает шум (стоп-слова и слова, которые встречаются 1 раз).

In [241]:
def tokens_lemmas(lst):
    if type(lst) == list:
        lst = ' '.join(lst)
    new_lst = preproc(lst)
    count = collections.Counter(new_lst).most_common()
    final_lst = []
    for c in count:
        if c[1] > 1:
            final_lst.append(c[0])
    sw = stopwords.words('russian')
    final_lst = [w for w in final_lst if w not in sw]
    return final_lst

In [242]:
good_list = set(tokens_lemmas(good_list))
bad_list = set(tokens_lemmas(bad_list))
both = good_list & bad_list
good_list = list(good_list - both)
bad_list = list(bad_list - both)

Функция определения тональности.

In [249]:
def tonality(sent):
    sent_lst = preproc(sent)
    k = 0
    for w in sent_lst:
        if w in good_list:
            k += 1
        elif w in bad_list:
            k -= 1
    if k > 0:
        return 1
    elif k < 0:
        return 0
    else:
        return randint(0, 1)

Подсчёт accuracy с помощью тестовой выборки (15 положительных и 15 отрицательных отзывов). **Итоговый результат: 0,8.**

In [264]:
real = []
test = []
for g in good_test:
    real.append(1)
    test.append(tonality(g))
for b in bad_test:
    real.append(0)
    test.append(tonality(b))
print('Accuracy =', round(accuracy_score(real, test), 2))

Accuracy = 0.8


### Улучшение программы

1. Можно смотреть не собственно слова, а их векторные представления (с помощью предобученной модели RusVectores, так как обучать модель для этой задачи незачем). Оценочная лексика может разбиться на группы в зависимоти от её тональности. В тестовой выборке мы могли бы сравнивать не слова, а векторые представления. Это было бы полезно в случае с синонимами, то есть, если у нас в списке есть слово "отвратительный", но нет "ужасный", то для нашей программы слово "ужасный" теряется, а вот векторные представления этих слов были бы достаточно близки, что помогло бы в опредлении тональности отзыва.
2. С помощью нейросетей можно провести оценку тональности эмодзи, которых, кстати, в отзывах очень много.